In [6]:
#installations
!pip install transformers
!pip install torch-geometric
!pip install pyarabic
!pip install arabert
!pip install transformers[tf-gpu]
!git clone https://github.com/aub-mind/arabert
!pip install -r arabert/requirements.txt
!pip install farasa
!pip install torch
!pip install dgl
!pip install stanza
!python -m spacy download en_core_web_sm

fatal: destination path 'arabert' already exists and is not an empty directory.


     ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
     --------------------------------------- 0.0/12.6 MB 145.2 kB/s eta 0:01:27
     --------------------------------------- 0.1/12.6 MB 252.2 kB/s eta 0:00:50
     --------------------------------------- 0.1/12.6 MB 327.7 kB/s eta 0:00:39
     --------------------------------------- 0.2/12.6 MB 482.7 kB/s eta 0:00:26
      -------------------------------------- 0.2/12.6 MB 570.5 kB/s eta 0:00:22
      -------------------------------------- 0.3/12.6 MB 707.1 kB/s eta 0:00:18
     - ------------------------------------- 0.4/12.6 MB 791.2 kB/s eta 0:00:16
     - ------------------------------------- 0.4/12.6 MB 839.7 kB/s eta 0:00:15
     - ------------------------------------- 0.5/12.6 MB 954.7 kB/s eta 

In [7]:
# nlp = stanza.Pipeline('ar')  # Initialize the Arabic pipeline for dependency parsing
import pandas as pd
import torch
from transformers import AutoModel, AutoTokenizer
from arabert.preprocess import ArabertPreprocessor
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from farasa.pos import FarasaPOSTagger
import dgl
import dgl.function as fn
from torch.utils.data import Dataset, DataLoader
from torch.nn import Module, Linear, LSTM
from dgl.nn.pytorch import GATConv
import spacy
import warnings
import stanza
import torch
import spacy
import numpy as np
import transformers
from sklearn.model_selection import train_test_split

model_name = 'aubmindlab/bert-base-arabertv01'
arabert_prep = ArabertPreprocessor(model_name=model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)



In [ ]:
import spacy
warnings.filterwarnings("ignore")
def get_dependency_tree(tokens):
  doc = nlp(' '.join(tokens))
  return [(token.dep_, token.head.i, token.i) for token in doc]

# Read training data
train_dataset = pd.read_csv('My_AR_HOTE_SB1_TRAIN (1).csv')
train_dataset.dropna(subset=['Polarity'], inplace=True)
train_dataset['Polarity'] = train_dataset['Polarity'].map({'positive': 0, 'negative': 1, 'neutral': 2})
train_dataset = train_dataset[:200]

test_dataset = pd.read_csv('My_AR_HOTE_SB1_TEST (1).csv')
test_dataset.dropna(subset=['Polarity'], inplace=True)
test_dataset['Polarity'] = test_dataset['Polarity'].map({'positive': 0, 'negative': 1, 'neutral': 2})
# test_dataset = test_dataset[:16]

import torch.nn.functional as F
nlp = spacy.load('en_core_web_sm')


class CustomDataset():
    def __init__(self, X,y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        # Get the text, term, and polarity
        text = self.X.iloc[index]['Text']
        term = self.X.iloc[index]['Term']
        polarity = self.y.iloc[index]

        # Preprocess text and term
        text = arabert_prep.preprocess(text)
        term = arabert_prep.preprocess(term)

        return text , term , polarity

class TwoDirectionalModel(Module):
    def __init__(self,batch_size, text_embedding_dim, term_embedding_dim, lstm_hidden_dim, gat_hidden_dim, num_classes):
        super().__init__()

        self.gat_hidden_dim = gat_hidden_dim

        # Term input layers
        self.term_lstm = nn.LSTM(term_embedding_dim, lstm_hidden_dim, batch_first=True, bidirectional=True)
        self.term_fc = Linear(2*lstm_hidden_dim, gat_hidden_dim)

        # Context input layers
        self.text_lstm = nn.LSTM(text_embedding_dim, lstm_hidden_dim, batch_first=True, bidirectional=True)
        self.text_fc = Linear(2*lstm_hidden_dim, gat_hidden_dim)

        # GAT layer for text data
        self.text_gat = GATConv(in_feats=gat_hidden_dim,
                                out_feats=512,
                                num_heads=batch_size)

        # GAT layer for term data
        self.term_gat = GATConv(in_feats=gat_hidden_dim,
                                out_feats=512,
                                num_heads=batch_size)

        # Output layer
        self.fc = Linear(1024, num_classes)

    def forward(self, train_term_outputs, train_text_outputs, text_token, term_token):
        text_dep = []
        for i in text_token:
            text_g = dgl.DGLGraph()
            text_g.add_nodes(len(i))
            text_dependency_graph = get_dependency_tree(i)
            parent, child = zip(*[(p, c) for r, p, c in text_dependency_graph])
            text_g.add_edges(parent, child)
            text_dep.append(text_g)

        term_dep = []
        for i in term_token:
            term_g = dgl.DGLGraph()
            term_g.add_nodes(len(i))
            term_dependency_graph = get_dependency_tree(i)
            parent, child = zip(*[(p, c) for r, p, c in term_dependency_graph])
            term_g.add_edges(parent, child)
            term_dep.append(term_g)

        
        # Term input LSTM
        train_term_outputs = torch.cat(train_term_outputs, dim=0)
        term_outputs, _ = self.term_lstm(train_term_outputs)
        term_outputs = self.term_fc(term_outputs)

        # Text input LSTM
        train_text_outputs = torch.cat(train_text_outputs, dim=0)
        text_outputs, _ = self.text_lstm(train_text_outputs)
        text_outputs = self.text_fc(text_outputs)

        # Text GAT layer
        text_dep = dgl.batch(text_dep)

        text_outputs = self.text_gat(text_dep, text_outputs)

        # Term GAT layer
        term_dep = dgl.batch(term_dep)
        term_outputs = self.term_gat(term_dep, term_outputs)

        # Graph mean node representation
        context_outputs = torch.mean(text_outputs, dim=0 )
        term_outputs = torch.mean(term_outputs, dim=0)
        # Concatenate text and term GAT outputs and use as input to output layer
        outputs = torch.cat([term_outputs, context_outputs], dim=1)
        outputs = self.fc(outputs)

        return outputs
    
X_train, X_val, y_train, y_val = train_test_split(train_dataset.loc[:,['Text','Term']], train_dataset['Polarity'], test_size=0.2, random_state=42)
X_test, y_text = test_dataset.loc[:,['Text','Term']],test_dataset['Polarity']
# Create dataloaders for training and testing data
train_data = CustomDataset(X_train,y_train)
val_data = CustomDataset(X_val,y_val)
test_data = CustomDataset(X_test, y_text)
train_dataloader = DataLoader(train_data, batch_size=16)
val_dataloader = DataLoader(val_data, batch_size=16)
test_dataloader = DataLoader(test_data, batch_size=16)

# Define the model, loss function, and optimizer
model = TwoDirectionalModel(batch_size=16,term_embedding_dim=300, text_embedding_dim=300, lstm_hidden_dim=128, gat_hidden_dim=256, num_classes=3)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 2
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for text_outputs,term_outputs , polarity in train_dataloader:
        text_token_list= []
        term_token_list =[]
        term_outputs_emp=[]
        text_outputs_emp=[]
        # Define a dictionary to map out-of-vocabulary tokens to a special <unk> token
        unk_token = "<unk>"
        text_vocab = {unk_token: 0} # initialize with <unk> token
        text_num_embeddings = 1
        for text in text_outputs:
            text_token = [token.text for token in nlp(text)][:5]
            # Add new word to `text_vocab` if not already there
            for token in text_token:
                if token not in text_vocab:
                    text_vocab[token] = text_num_embeddings
                    text_num_embeddings += 1
            text_word_emb = nn.Embedding(num_embeddings=len(text_vocab), embedding_dim=300)
            # Map out-of-vocabulary tokens to <unk> token
            text_input = [text_vocab[token] if token in text_vocab else text_vocab[unk_token] for token in text_token]
            text_input_emb = text_word_emb(torch.LongTensor(text_input))
            text_token_list.append(text_token)
            text_outputs_emp.append(text_input_emb)
        term_vocab = {unk_token: 0} # initialize with <unk> token
        term_num_embeddings = 1
        for term in term_outputs:
            term_token = [token.text for token in nlp(term)][:1]
            # Add new word to `text_vocab` if not already there
            for token in term_token:
                if token not in term_vocab:
                    term_vocab[token] = term_num_embeddings
                    term_num_embeddings += 1
            term_word_emb = nn.Embedding(num_embeddings=len(term_vocab), embedding_dim=300)
            # Map out-of-vocabulary tokens to <unk> token
            term_input = [term_vocab[token] if token in term_vocab else term_vocab[unk_token] for token in term_token]
            term_input_emb = term_word_emb(torch.LongTensor(term_input))
            term_token_list.append(term_token)
            term_outputs_emp.append(term_input_emb)

        # Call the model forward method with concatenated tensors
        logits = model( term_outputs_emp, text_outputs_emp, text_token_list, term_token_list)
        # Compute logits and loss
        train_polarity=polarity
        if(logits.shape[0] == train_polarity.shape[0]) :
            loss = criterion(logits, train_polarity.long())

            # Backpropagation and weight updates
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

    # Evaluate the model on the test set
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for text_outputs , term_outputs , polarity in val_dataloader:
            test_polarity=polarity
            # Compute logits and accuracy
            text_token_list= []
            term_token_list =[]
            term_outputs_emp=[]
            text_outputs_emp=[]
            # Define a dictionary to map out-of-vocabulary tokens to a special <unk> token
            unk_token = "<unk>"
            text_vocab = {unk_token: 0} # initialize with <unk> token
            text_num_embeddings = 1
            for text in text_outputs:
                text_token = [token.text for token in nlp(text)][:5]
                # Add new word to `text_vocab` if not already there
                for token in text_token:
                    if token not in text_vocab:
                        text_vocab[token] = text_num_embeddings
                        text_num_embeddings += 1
                text_word_emb = nn.Embedding(num_embeddings=len(text_vocab), embedding_dim=300)
                # Map out-of-vocabulary tokens to <unk> token
                text_input = [text_vocab[token] if token in text_vocab else text_vocab[unk_token] for token in text_token]
                text_input_emb = text_word_emb(torch.LongTensor(text_input))
                text_token_list.append(text_token)
                text_outputs_emp.append(text_input_emb)
            term_vocab = {unk_token: 0} # initialize with <unk> token
            term_num_embeddings = 1
            for term in term_outputs:
                term_token = [token.text for token in nlp(term)][:1]
                # Add new word to `text_vocab` if not already there
                for token in term_token:
                    if token not in term_vocab:
                        term_vocab[token] = term_num_embeddings
                        term_num_embeddings += 1
                term_word_emb = nn.Embedding(num_embeddings=len(term_vocab), embedding_dim=300)
                # Map out-of-vocabulary tokens to <unk> token
                term_input = [term_vocab[token] if token in term_vocab else term_vocab[unk_token] for token in term_token]
                term_input_emb = term_word_emb(torch.LongTensor(term_input))
                term_token_list.append(term_token)
                term_outputs_emp.append(term_input_emb)
            logits = model(term_outputs_emp, text_outputs_emp, text_token_list, term_token_list)
            if (test_polarity.shape[0] == logits.shape[0]):
                _, predicted = torch.max(logits.data, 1)
                total += test_polarity.size(0)
                correct += (predicted == test_polarity).sum().item()
    print(f"Epoch {epoch+1}: Train Loss = {running_loss/len(train_dataloader):.3f}, Validation Accuracy = {(100*correct/total):.2f}%")


Epoch 1: Train Loss = 0.955, Validation Accuracy = 71.88%
Epoch 2: Train Loss = 0.734, Validation Accuracy = 65.62%


## Model Testing 

In [5]:
from sklearn.metrics import classification_report
# Set model to evaluation mode
model.eval()
# Initialize variables for accuracy computation
y_label = []
y_predict = []
correct = 0
total = 0
# Disable gradient computation for testing
with torch.no_grad():

    # Loop through batches in dataloader
    for text_outputs, term_outputs, polarity in test_dataloader:
        
        # Preprocess inputs (tokenize and embed)
        text_token_list = []
        term_token_list = []
        term_outputs_emp = []
        text_outputs_emp = []
        unk_token = "<unk>"
        for text in text_outputs:
            text_token = [token.text for token in nlp(text)][:5]
            for token in text_token:
                if token not in text_vocab:
                    text_vocab[token] = text_num_embeddings
                    text_num_embeddings += 1
            text_word_emb = nn.Embedding(num_embeddings=len(text_vocab), embedding_dim=300)
            text_input = [text_vocab[token] if token in text_vocab else text_vocab[unk_token] for token in text_token]
            text_input_emb = text_word_emb(torch.LongTensor(text_input))
            text_token_list.append(text_token)
            text_outputs_emp.append(text_input_emb)
        term_vocab = {unk_token: 0}
        term_num_embeddings = 1
        for term in term_outputs:
            term_token = [token.text for token in nlp(term)][:1]
            for token in term_token:
                if token not in term_vocab:
                    term_vocab[token] = term_num_embeddings
                    term_num_embeddings += 1
            term_word_emb = nn.Embedding(num_embeddings=len(term_vocab), embedding_dim=300)
            term_input = [term_vocab[token] if token in term_vocab else term_vocab[unk_token] for token in term_token]
            term_input_emb = term_word_emb(torch.LongTensor(term_input))
            term_token_list.append(term_token)
            term_outputs_emp.append(term_input_emb)

        # Call forward method and get logits
        logits = model(term_outputs_emp, text_outputs_emp, text_token_list, term_token_list)
        test_polarity = polarity
        if (test_polarity.shape[0] == logits.shape[0]):
            # Compute accuracy
            _, predicted = torch.max(logits.data, 1)
            total += test_polarity.size(0)
            correct += (predicted == test_polarity).sum().item()
            y_label.extend(test_polarity)
            y_predict.extend(predicted)
# Print accuracy
print(f"Test Accuracy = {(100*correct/total):.2f}%")
plot_labels = ['positive', 'negative', 'neutral']
report = classification_report(y_label, y_predict, target_names=plot_labels)
print(report)

Test Accuracy = 54.55%
              precision    recall  f1-score   support

    positive       0.57      0.88      0.69      1500
    negative       0.33      0.11      0.16       923
     neutral       0.00      0.00      0.00       169

    accuracy                           0.55      2592
   macro avg       0.30      0.33      0.29      2592
weighted avg       0.45      0.55      0.46      2592



In [ ]:
from nltk.parse.stanford import StanfordDependencyParser
stanford_parser_dir = "C:/Users/hP/Desktop/lo/stanford-corenlp-4.2.0-models-arabic.jar"
stanford_model_dir = "C:/Users/hP/Desktop/lo/stanford-parser-full-2020-11-17/stanford-parser.jar"
java_options = '-mx1g'
stanford_parser = StanfordDependencyParser(
    stanford_parser_dir,
    stanford_model_dir,
    java_options=java_options
)

def procc(text):
    parsed_sents = stanford_parser.raw_parse(text)
    dep_tree = next(parsed_sents).to_conll(4)
    g = dgl.DGLGraph()
    words = []
    deps = []
    labels = {}
    for line in dep_tree.split('\n'):
        parts = line.strip().split('\t')
        if len(parts) != 4:
            continue
        word = parts[0]
        pos = parts[1]
        relation = parts[2]
        if relation not in labels:
            labels[relation] = len(labels)
        try:
            parent_idx = int(parts[3]) - 1
        except ValueError:
            parent_idx = -1
        words.append(word)
        deps.append((parent_idx, len(words) - 1, labels[relation]))  # Use the label instead of the string relation
    g.add_nodes(len(words))
    src, dst, rel = zip(*deps)
    g.add_edges(src, dst)
    g.edata['rel'] = torch.tensor(rel)

    return words ,g

train_dataset = pd.read_csv('My_AR_HOTE_SB1_TRAIN (1).csv')
train_dataset.dropna(subset=['Polarity'], inplace=True)
train_dataset['Polarity'] = train_dataset['Polarity'].map({'positive': 0, 'negative': 1, 'neutral': 2})
train_dataset = train_dataset[:200]

test_dataset = pd.read_csv('My_AR_HOTE_SB1_TEST (1).csv')
test_dataset.dropna(subset=['Polarity'], inplace=True)
test_dataset['Polarity'] = test_dataset['Polarity'].map({'positive': 0, 'negative': 1, 'neutral': 2})
test_dataset = test_dataset[:16]


class CustomDataset():
    def __init__(self, X,y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        # Get the text, term, and polarity
        text = self.X.iloc[index]['Text']
        term = self.X.iloc[index]['Term']
        polarity = self.y.iloc[index]

        # Preprocess text and term
        text = arabert_prep.preprocess(text)
        term = arabert_prep.preprocess(term)

        return text , term , polarity

class TwoDirectionalModel(Module):
    def __init__(self,batch_size, text_embedding_dim, term_embedding_dim, lstm_hidden_dim, gat_hidden_dim, num_classes):
        super().__init__()

        self.gat_hidden_dim = gat_hidden_dim

        # Term input layers
        self.term_lstm = nn.LSTM(term_embedding_dim, lstm_hidden_dim, batch_first=True, bidirectional=True)
        self.term_fc = Linear(2*lstm_hidden_dim, gat_hidden_dim)

        # Context input layers
        self.text_lstm = nn.LSTM(text_embedding_dim, lstm_hidden_dim, batch_first=True, bidirectional=True)
        self.text_fc = Linear(2*lstm_hidden_dim, gat_hidden_dim)

        # GAT layer for text data
        self.text_gat = GATConv(in_feats=gat_hidden_dim,
                                out_feats=512,
                                num_heads=batch_size)

        # GAT layer for term data
        self.term_gat = GATConv(in_feats=gat_hidden_dim,
                                out_feats=512,
                                num_heads=batch_size)

        # Output layer
        self.fc = Linear(1024, num_classes)

    def forward(self, train_term_outputs, train_text_outputs, text_dep, term_dep):        
        # Term input LSTM
        train_term_outputs = torch.cat(train_term_outputs, dim=0)
        term_outputs, _ = self.term_lstm(train_term_outputs)
        term_outputs = self.term_fc(term_outputs)
        # Text input LSTM
        train_text_outputs = torch.cat(train_text_outputs, dim=0)
        text_outputs, _ = self.text_lstm(train_text_outputs)
        text_outputs = self.text_fc(text_outputs)
        print("g")
        # Text GAT layer
        text_dep = dgl.batch(text_dep)
        print(text_dep)
        print(text_outputs)
        text_outputs = self.text_gat(text_dep, text_outputs)
        # Term GAT layer
        term_dep = dgl.batch(term_dep)
        term_outputs = self.term_gat(term_dep, term_outputs)

        # Graph mean node representation
        context_outputs = torch.mean(text_outputs, dim=0 )
        term_outputs = torch.mean(term_outputs, dim=0)
        # Concatenate text and term GAT outputs and use as input to output layer
        outputs = torch.cat([term_outputs, context_outputs], dim=1)
        outputs = self.fc(outputs)

        return outputs

# Create dataloaders for training and testing data

X_train, X_val, y_train, y_val = train_test_split(train_dataset.loc[:,['Text','Term']], train_dataset['Polarity'], test_size=0.2, random_state=42)
X_test, y_text = test_dataset.loc[:,['Text','Term']],test_dataset['Polarity']
# Create dataloaders for training and testing data
train_data = CustomDataset(X_train,y_train)
val_data = CustomDataset(X_val,y_val)
test_data = CustomDataset(X_test, y_text)
train_dataloader = DataLoader(train_data, batch_size=16)
val_dataloader = DataLoader(val_data, batch_size=16)
test_dataloader = DataLoader(test_data, batch_size=16)
# Define the model, loss function, and optimizer
tokenizer.model_max_length = 300
model = TwoDirectionalModel(batch_size=16,term_embedding_dim=tokenizer.model_max_length, text_embedding_dim=tokenizer.model_max_length, lstm_hidden_dim=128, gat_hidden_dim=256, num_classes=3)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for text_outputs, term_outputs , polarity in train_dataloader:
        # Check that the size of text_outputs and term_outputs are equal
        if len(text_outputs) != len(term_outputs):
            continue
        text_dep_list = []
        term_dep_list = []
        term_outputs_emp = []
        text_outputs_emp = []
        for i in range(len(text_outputs)):
            text = text_outputs[i]
            term = term_outputs[i]
            unk_token = "<unk>"
            text_vocab = {unk_token: 0} # initialize with <unk> token
            text_num_embeddings = 1
            text,text_dep= procc(text)
            term,term_dep=procc(term)
            for token in text:
                if token not in text_vocab:
                    text_vocab[token] = text_num_embeddings
                    text_num_embeddings += 1
            text_word_emb = nn.Embedding(num_embeddings=len(text_vocab), embedding_dim=300)
            # Map out-of-vocabulary tokens to <unk> token
            text_input = [text_vocab[token] if token in text_vocab else text_vocab[unk_token] for token in text]
            text_input_emb = text_word_emb(torch.LongTensor(text_input))
            text_dep_list.append(text_dep)
            text_outputs_emp.append(text_input_emb)
            unk_token = "<unk>"
            term_vocab = {unk_token: 0} # initialize with <unk> token
            term_num_embeddings = 1
            for token in term:
                if token not in term_vocab:
                    term_vocab[token] = term_num_embeddings
                    term_num_embeddings += 1
            term_word_emb = nn.Embedding(num_embeddings=len(term_vocab), embedding_dim=300)
            # Map out-of-vocabulary tokens to <unk> token
            term_input = [term_vocab[token] if token in term_vocab else term_vocab[unk_token] for token in term]
            term_input_emb = term_word_emb(torch.LongTensor(term_input))
            term_dep_list.append(term_dep)
            term_outputs_emp.append(term_input_emb)
        # Call the model forward method with concatenated tensors
        logits = model(term_outputs_emp, text_outputs_emp, text_dep_list, term_dep_list)
        # Compute logits and loss
        train_polarity = polarity
        if(logits.shape[0] == train_polarity.shape[0]) :
          loss = criterion(logits, train_polarity.long())
          # Backpropagation and weight updates
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          running_loss += loss.item()

    # Evaluate the model on the test set
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for text_outputs , term_outputs , polarity in val_dataloader:        
          text_dep_list = []
          term_dep_list = []
          term_outputs_emp = []
          text_outputs_emp = []
          for i in range(len(text_outputs)):
              text = text_outputs[i]
              term = term_outputs[i]
              unk_token = "<unk>"
              text_vocab = {unk_token: 0} # initialize with <unk> token
              text_num_embeddings = 1
              text,text_dep= procc(text)
              term,term_dep=procc(term)
              for token in text:
                  if token not in text_vocab:
                      text_vocab[token] = text_num_embeddings
                      text_num_embeddings += 1
              text_word_emb = nn.Embedding(num_embeddings=len(text_vocab), embedding_dim=300)
            # Map out-of-vocabulary tokens to <unk> token
              text_input = [text_vocab[token] if token in text_vocab else text_vocab[unk_token] for token in text]
              text_input_emb = text_word_emb(torch.LongTensor(text_input))
              text_dep_list.append(text_dep)
              text_outputs_emp.append(text_input_emb)
              unk_token = "<unk>"
              term_vocab = {unk_token: 0} # initialize with <unk> token
              term_num_embeddings = 1
              for token in term:
                  if token not in term_vocab:
                      term_vocab[token] = term_num_embeddings
                      term_num_embeddings += 1
              term_word_emb = nn.Embedding(num_embeddings=len(term_vocab), embedding_dim=300)
            # Map out-of-vocabulary tokens to <unk> token
              term_input = [term_vocab[token] if token in term_vocab else term_vocab[unk_token] for token in term]
              term_input_emb = term_word_emb(torch.LongTensor(term_input))
              term_dep_list.append(term_dep)
              term_outputs_emp.append(term_input_emb)
          logits = model(term_outputs_emp, text_outputs_emp, text_dep_list, term_dep_list)
          test_polarity = polarity 
          if (test_polarity.shape[0] == logits.shape[0]):
              _, predicted = torch.max(logits.data, 1)
              total += test_polarity.size(0)
              correct += (predicted == test_polarity).sum().item()
    print(f"Epoch {epoch+1}: Train Loss = {running_loss/len(train_dataloader):.3f}, Validation Accuracy = {(100*correct/total):.2f}%")

## Model Testing 

In [ ]:
from sklearn.metrics import classification_report
# Set model to evaluation mode
model.eval()
# Initialize variables for accuracy computation
y_label = []
y_predict = []
correct = 0
total = 0
# Disable gradient computation for testing
with torch.no_grad():

    # Loop through batches in dataloader
    for text_outputs, term_outputs, polarity in test_dataloader:
        text_dep_list = []
        term_dep_list = []
        term_outputs_emp = []
        text_outputs_emp = []
        for i in range(len(text_outputs)):
            text = text_outputs[i]
            term = term_outputs[i]
            unk_token = "<unk>"
            text_vocab = {unk_token: 0} # initialize with <unk> token
            text_num_embeddings = 1
            text,text_dep= procc(text)
            term,term_dep=procc(term)
            for token in text:
                if token not in text_vocab:
                    text_vocab[token] = text_num_embeddings
                    text_num_embeddings += 1
            text_word_emb = nn.Embedding(num_embeddings=len(text_vocab), embedding_dim=300)
        # Map out-of-vocabulary tokens to <unk> token
            text_input = [text_vocab[token] if token in text_vocab else text_vocab[unk_token] for token in text]
            text_input_emb = text_word_emb(torch.LongTensor(text_input))
            text_dep_list.append(text_dep)
            text_outputs_emp.append(text_input_emb)
            unk_token = "<unk>"
            term_vocab = {unk_token: 0} # initialize with <unk> token
            term_num_embeddings = 1
            for token in term:
                if token not in term_vocab:
                    term_vocab[token] = term_num_embeddings
                    term_num_embeddings += 1
            term_word_emb = nn.Embedding(num_embeddings=len(term_vocab), embedding_dim=300)
        # Map out-of-vocabulary tokens to <unk> token
            term_input = [term_vocab[token] if token in term_vocab else term_vocab[unk_token] for token in term]
            term_input_emb = term_word_emb(torch.LongTensor(term_input))
            term_dep_list.append(term_dep)
            term_outputs_emp.append(term_input_emb)
        logits = model(term_outputs_emp, text_outputs_emp, text_dep_list, term_dep_list)
        test_polarity = polarity 
        if (test_polarity.shape[0] == logits.shape[0]):
            _, predicted = torch.max(logits.data, 1)
            total += test_polarity.size(0)
            correct += (predicted == test_polarity).sum().item()
            y_label.extend(test_polarity)
            y_predict.extend(predicted)
# Print accuracy
print(f"Test Accuracy = {(100*correct/total):.2f}%")
plot_labels = ['positive', 'negative', 'neutral']
report = classification_report(y_label, y_predict, target_names=plot_labels)
print(report)

In [6]:
import nltk
nltk.download('punkt')
# Define a function to get the dependency tree of a sentence using StanfordNLP
nlp = stanza.Pipeline(lang='ar', processors='tokenize, lemma, pos, depparse', tokenize_pretokenized=True , tokenize_batch_size=128, tokenize_max_length=5)
def get_dependency_tree(tokens):
    text = ' '.join(tokens)
    doc = nlp(text)
    tree = [(word.deprel, word.head - 1, word.id - 1) for sentence in doc.sentences for word in sentence.words]
    return tree


def tokenize_text_arabic(text, max_len=5):
    tokens = nltk.word_tokenize(text)
    if len(tokens) > max_len:
        tokens = tokens[:max_len]
    return tokens


# Read training and testing data
train_dataset = pd.read_csv('My_AR_HOTE_SB1_TRAIN (1).csv')
train_dataset.dropna(subset=['Polarity'], inplace=True)
train_dataset['Polarity'] = train_dataset['Polarity'].map({'positive': 0, 'negative': 1, 'neutral': 2})
train_dataset = train_dataset[:32]

test_dataset = pd.read_csv('My_AR_HOTE_SB1_TEST (1).csv')
test_dataset.dropna(subset=['Polarity'], inplace=True)
test_dataset['Polarity'] = test_dataset['Polarity'].map({'positive': 0, 'negative': 1, 'neutral': 2})
test_dataset = test_dataset[:16]

# Initialize Farasa POS tagger
pos_tagger = FarasaPOSTagger()

class CustomDataset():
    def __init__(self, X,y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        # Get the text, term, and polarity
        text = self.X.iloc[index]['Text']
        term = self.X.iloc[index]['Term']
        polarity = self.y.iloc[index]

        # Preprocess text and term
        text = arabert_prep.preprocess(text)
        term = arabert_prep.preprocess(term)

        return text , term , polarity


class TwoDirectionalModel(Module):
    def __init__(self,batch_size, text_embedding_dim, term_embedding_dim, lstm_hidden_dim, gat_hidden_dim, num_classes):
        super().__init__()

        self.gat_hidden_dim = gat_hidden_dim

        # Term input layers
        self.term_lstm = nn.LSTM(term_embedding_dim, lstm_hidden_dim, batch_first=True, bidirectional=True)
        self.term_fc = Linear(2*lstm_hidden_dim, gat_hidden_dim)

        # Context input layers
        self.text_lstm = nn.LSTM(text_embedding_dim, lstm_hidden_dim, batch_first=True, bidirectional=True)
        self.text_fc = Linear(2*lstm_hidden_dim, gat_hidden_dim)

        # GAT layer for text data
        self.text_gat = GATConv(in_feats=gat_hidden_dim,
                                out_feats=512,
                                num_heads=batch_size)

        # GAT layer for term data
        self.term_gat = GATConv(in_feats=gat_hidden_dim,
                                out_feats=512,
                                num_heads=batch_size)

        # Output layer
        self.fc = Linear(1024, num_classes)

    def forward(self, train_term_outputs, train_text_outputs, text_token, term_token):
        text_dep = []
        for i in text_token:
            text_g = dgl.DGLGraph()
            text_g.add_nodes(len(i))
            text_dependency_graph = get_dependency_tree(i)
            parent, child = zip(*[(p, c) for r, p, c in text_dependency_graph])
            text_g.add_edges(parent, child)
            text_dep.append(text_g)

        term_dep = []
        for i in term_token:
            term_g = dgl.DGLGraph()
            term_g.add_nodes(len(i))
            term_dependency_graph = get_dependency_tree(i)
            parent, child = zip(*[(p, c) for r, p, c in term_dependency_graph])
            term_g.add_edges(parent, child)
            term_dep.append(term_g)
        
        # Term input LSTM
        train_term_outputs = torch.cat(train_term_outputs, dim=0)
        term_outputs, _ = self.term_lstm(train_term_outputs)
        term_outputs = self.term_fc(term_outputs)
        # Text input LSTM
        train_text_outputs = torch.cat(train_text_outputs, dim=0)
        text_outputs, _ = self.text_lstm(train_text_outputs)
        text_outputs = self.text_fc(text_outputs)
        # Text GAT layer
        text_dep = dgl.batch(text_dep)
        text_outputs = self.text_gat(text_dep, text_outputs)
        # Term GAT layer
        term_dep = dgl.batch(term_dep)
        term_outputs = self.term_gat(term_dep, term_outputs)

        # Graph mean node representation
        context_outputs = torch.mean(text_outputs, dim=0 )
        term_outputs = torch.mean(term_outputs, dim=0)
        # Concatenate text and term GAT outputs and use as input to output layer
        outputs = torch.cat([term_outputs, context_outputs], dim=1)
        outputs = self.fc(outputs)

        return outputs

X_train, X_val, y_train, y_val = train_test_split(train_dataset.loc[:,['Text','Term']], train_dataset['Polarity'], test_size=0.2, random_state=42)
X_test, y_text = test_dataset.loc[:,['Text','Term']],test_dataset['Polarity']
# Create dataloaders for training and testing data
train_data = CustomDataset(X_train,y_train)
val_data = CustomDataset(X_val,y_val)
test_data = CustomDataset(X_test, y_text)
train_dataloader = DataLoader(train_data, batch_size=16)
val_dataloader = DataLoader(val_data, batch_size=16)
test_dataloader = DataLoader(test_data, batch_size=16)
# Define the model, loss function, and optimizer
tokenizer.model_max_length = 300
print(tokenizer.model_max_length)
model = TwoDirectionalModel(batch_size=16,term_embedding_dim=tokenizer.model_max_length, text_embedding_dim=tokenizer.model_max_length, lstm_hidden_dim=128, gat_hidden_dim=256, num_classes=3)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for text_outputs, term_outputs, Polarity in train_dataloader:
        # Check that the size of text_outputs and term_outputs are equal
        if len(text_outputs) != len(term_outputs):
            continue
        text_token_list = []
        term_token_list = []
        term_outputs_emp = []
        text_outputs_emp = []
        for i in range(len(text_outputs)):
            text = text_outputs[i]
            term = term_outputs[i]
            unk_token = "<unk>"
            text_vocab = {unk_token: 0} # initialize with <unk> token
            text_num_embeddings = 1
            text= tokenize_text_arabic(text, max_len=5)
            term=tokenize_text_arabic(term, max_len=1)
            for token in text:
                if token not in text_vocab:
                    text_vocab[token] = text_num_embeddings
                    text_num_embeddings += 1
            text_word_emb = nn.Embedding(num_embeddings=len(text_vocab), embedding_dim=300)
            # Map out-of-vocabulary tokens to <unk> token
            text_input = [text_vocab[token] if token in text_vocab else text_vocab[unk_token] for token in text]
            text_input_emb = text_word_emb(torch.LongTensor(text_input))
            text_token_list.append(text)
            text_outputs_emp.append(text_input_emb)
            unk_token = "<unk>"
            term_vocab = {unk_token: 0} # initialize with <unk> token
            term_num_embeddings = 1
            for token in term:
                if token not in term_vocab:
                    term_vocab[token] = term_num_embeddings
                    term_num_embeddings += 1
            term_word_emb = nn.Embedding(num_embeddings=len(term_vocab), embedding_dim=300)
            # Map out-of-vocabulary tokens to <unk> token
            term_input = [term_vocab[token] if token in term_vocab else term_vocab[unk_token] for token in term]
            term_input_emb = term_word_emb(torch.LongTensor(term_input))
            term_token_list.append(term)
            term_outputs_emp.append(term_input_emb)
        # Call the model forward method with concatenated tensors
        logits = model(term_outputs_emp, text_outputs_emp, text_token_list, term_token_list)
        # Compute logits and loss
        train_polarity = Polarity
        if(logits.shape[0] == train_polarity.shape[0]) :
          loss = criterion(logits, train_polarity.long())
          # Backpropagation and weight updates
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          running_loss += loss.item()

    # Evaluate the model on the test set
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for text_outputs , term_outputs , Polarity in val_dataloader:
          text_token_list = []
          term_token_list = []
          term_outputs_emp = []
          text_outputs_emp = []
          for i in range(len(text_outputs)):
            text = text_outputs[i]
            term = term_outputs[i]
            unk_token = "<unk>"
            text_vocab = {unk_token: 0} # initialize with <unk> token
            text_num_embeddings = 1
            text= tokenize_text_arabic(text, max_len=5)
            term=tokenize_text_arabic(term, max_len=1)
            for token in text:
                if token not in text_vocab:
                    text_vocab[token] = text_num_embeddings
                    text_num_embeddings += 1
            text_word_emb = nn.Embedding(num_embeddings=len(text_vocab), embedding_dim=300)
            # Map out-of-vocabulary tokens to <unk> token
            text_input = [text_vocab[token] if token in text_vocab else text_vocab[unk_token] for token in text]
            text_input_emb = text_word_emb(torch.LongTensor(text_input))
            text_token_list.append(text)
            text_outputs_emp.append(text_input_emb)
            unk_token = "<unk>"
            term_vocab = {unk_token: 0} # initialize with <unk> token
            term_num_embeddings = 1
            for token in term:
                if token not in term_vocab:
                    term_vocab[token] = term_num_embeddings
                    term_num_embeddings += 1
            term_word_emb = nn.Embedding(num_embeddings=len(term_vocab), embedding_dim=300)
            # Map out-of-vocabulary tokens to <unk> token
            term_input = [term_vocab[token] if token in term_vocab else term_vocab[unk_token] for token in term]
            term_input_emb = term_word_emb(torch.LongTensor(term_input))
            term_token_list.append(term)
            term_outputs_emp.append(term_input_emb)
          logits = model(term_outputs_emp, text_outputs_emp, text_token_list, term_token_list)
          test_polarity = Polarity
          if (test_polarity.shape[0] == logits.shape[0]):
              _, predicted = torch.max(logits.data, 1)
              total += test_polarity.size(0)
              correct += (predicted == test_polarity).sum().item()
    print(f"Epoch {epoch+1}: Train Loss = {running_loss/len(train_dataloader):.3f}, Validation Accuracy = {(100*correct/total):.2f}%")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2023-11-05 17:12:19 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2023-11-05 17:12:21 INFO: Loading these models for language: ar (Arabic):
| Processor | Package       |
-----------------------------
| tokenize  | padt          |
| pos       | padt_charlm   |
| lemma     | padt_nocharlm |
| depparse  | padt_charlm   |

2023-11-05 17:12:21 INFO: Using device: cpu
2023-11-05 17:12:21 INFO: Loading: tokenize
2023-11-05 17:12:21 INFO: Loading: pos
2023-11-05 17:12:22 INFO: Loading: lemma
2023-11-05 17:12:22 INFO: Loading: depparse
2023-11-05 17:12:23 INFO: Done loading processors!


300


c:\Users\hP\AppData\Local\Programs\Python\Python311\Lib\site-packages\dgl\heterograph.py:92: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning(


: 

## Model Testing 

In [ ]:
from sklearn.metrics import classification_report
# Set model to evaluation mode
model.eval()
# Initialize variables for accuracy computation
y_label = []
y_predict = []
correct = 0
total = 0
# Disable gradient computation for testing
with torch.no_grad():

    # Loop through batches in dataloader
    for text_outputs, term_outputs, polarity in test_dataloader:

        text_token_list = []
        term_token_list = []
        term_outputs_emp = []
        text_outputs_emp = []
        for i in range(len(text_outputs)):
            text = text_outputs[i]
            term = term_outputs[i]
            unk_token = "<unk>"
            text_vocab = {unk_token: 0} # initialize with <unk> token
            text_num_embeddings = 1
            text= tokenize_text_arabic(text, max_len=5)
            term=tokenize_text_arabic(term, max_len=1)
            for token in text:
                if token not in text_vocab:
                    text_vocab[token] = text_num_embeddings
                    text_num_embeddings += 1
            text_word_emb = nn.Embedding(num_embeddings=len(text_vocab), embedding_dim=300)
            # Map out-of-vocabulary tokens to <unk> token
            text_input = [text_vocab[token] if token in text_vocab else text_vocab[unk_token] for token in text]
            text_input_emb = text_word_emb(torch.LongTensor(text_input))
            text_token_list.append(text)
            text_outputs_emp.append(text_input_emb)
            unk_token = "<unk>"
            term_vocab = {unk_token: 0} # initialize with <unk> token
            term_num_embeddings = 1
            for token in term:
                if token not in term_vocab:
                    term_vocab[token] = term_num_embeddings
                    term_num_embeddings += 1
            term_word_emb = nn.Embedding(num_embeddings=len(term_vocab), embedding_dim=300)
            # Map out-of-vocabulary tokens to <unk> token
            term_input = [term_vocab[token] if token in term_vocab else term_vocab[unk_token] for token in term]
            term_input_emb = term_word_emb(torch.LongTensor(term_input))
            term_token_list.append(term)
            term_outputs_emp.append(term_input_emb)
        logits = model(term_outputs_emp, text_outputs_emp, text_token_list, term_token_list)
        test_polarity = polarity
        if (test_polarity.shape[0] == logits.shape[0]):
            _, predicted = torch.max(logits.data, 1)
            total += test_polarity.size(0)
            correct += (predicted == test_polarity).sum().item()
            y_label.extend(test_polarity)
            y_predict.extend(predicted)
# Print accuracy
print(f"Test Accuracy = {(100*correct/total):.2f}%")
plot_labels = ['positive', 'negative', 'neutral']
report = classification_report(y_label, y_predict, target_names=plot_labels)
print(report)